In [1]:
pip install numpy==1.23.0


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

C:\Users\97254\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\97254\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [3]:
#open all data and make a DF
df_train=pd.read_csv("TrainDataFrame_After_Cleaning.csv")
test_data = pd.read_csv('TestDataFrame_After_Cleaning.csv')


In [4]:

#we can see that each df have the same nums of features import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

data_train = df_train
data_test = test_data

# Define features and target
X_train = data_train.drop(columns=['FLOWCOOLPRESSURE'])
y_train = data_train['FLOWCOOLPRESSURE']

X_test = data_test.drop(columns=['FLOWCOOLPRESSURE'])
y_test = data_test['FLOWCOOLPRESSURE']



In [5]:
#make it all to the same type
print(X_train.dtypes)
print(y_train.dtype)
X_train = X_train.astype(float)
y_train = y_train.astype(float)
X_test=X_test.astype(float)
y_test=y_test.astype(float)

Unnamed: 0             float64
stage                  float64
Lot                    float64
recipe                 float64
IONGAUGEPRESSURE       float64
ETCHAUXSOURCETIMER     float64
ETCHAUX2SOURCETIMER    float64
ACTUALSTEPDURATION     float64
dtype: object
float64


In [6]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Lasso Regression': Lasso(),
    'Random Forest Regression': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting Regression': GradientBoostingRegressor(n_estimators=100, random_state=42),
}

# Dictionary to store the results
results = {}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {'Mean Squared Error': mse, 'R-squared': r2}

# Display the results
for name, result in results.items():
    print(f"{name}:")
    print(f"  Mean Squared Error: {result['Mean Squared Error']}")
    print(f"  R-squared: {result['R-squared']}")
    print()

# Identify the optimal algorithm based on Mean Squared Error
optimal_algorithm = min(results, key=lambda x: results[x]['Mean Squared Error'])
print(f"Optimal Algorithm: {optimal_algorithm}")

# Use the optimal model to make predictions on your test data
optimal_model = models[optimal_algorithm]
test_predictions = optimal_model.predict(X_test)  # Use `X_test` or your actual test features DataFrame

# Print test predictions
print("Test Predictions:")
print(test_predictions)

# Calculate and print the R-squared value for the optimal model
r2_optimal = r2_score(y_test, test_predictions)
print(f"R-squared for {optimal_algorithm}: {r2_optimal:.4f}")

# Display some actual vs predicted values for better understanding
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': test_predictions})
print(comparison_df.head(10))  # Display first 10 rows of the comparison

Linear Regression:
  Mean Squared Error: 0.0523958127420541
  R-squared: -10.815178502940107

Lasso Regression:
  Mean Squared Error: 0.004580018458250702
  R-squared: -0.032787407982310945

Random Forest Regression:
  Mean Squared Error: 0.004301831458202787
  R-squared: 0.029943350273976765

Gradient Boosting Regression:
  Mean Squared Error: 0.11190696286398773
  R-squared: -24.2348550917443

Optimal Algorithm: Random Forest Regression
Test Predictions:
[0.15764223 0.15764223 0.00018004 ... 0.00994562 0.00994562 0.00994562]
R-squared for Random Forest Regression: 0.0299
     Actual  Predicted
0  0.142693   0.157642
1  0.142837   0.157642
2  0.143124   0.000180
3  0.143698   0.130143
4  0.143878   0.160134
5  0.144058   0.160134
6  0.144237   0.160134
7  0.144417   0.160134
8  0.144417   0.160134
9  0.144417   0.160134


In [7]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score

# Define polynomial features
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Assuming 'models' is a dictionary of models to evaluate
for name, model in models.items():
    # Use X_train_poly instead of X_train_scaled
    scores = cross_val_score(model, X_train_poly, y_train, cv=5, scoring='neg_mean_squared_error')
    print(f"{name} Cross-Validation Mean Squared Error: {-scores.mean()}")


Linear Regression Cross-Validation Mean Squared Error: 61611350764.166
Lasso Regression Cross-Validation Mean Squared Error: 0.0061167540272050085


KeyboardInterrupt: 

In [ ]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'alpha': [0.1, 0.5, 1.0, 10.0],
#     'max_iter': [1000, 2000]
# }

# grid_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(X_train_scaled, y_train)
# print(f"Best Lasso Params: {grid_search.best_params_}")
#Best Lasso Params: {'alpha': 0.1, 'max_iter': 1000}



#after Hyperparameter Tuning with lasso and cross validtion this is the best model of the lasso: alpha= 0.1, max_iter= 1000


In [ ]:
#our pc can't allocate this big data so this is our best model: Random Forest Regression